## Coupled Markov Process on Pitch and Rhythm

## 

In [1]:
# import data and packages
import numpy as np
from fractions import Fraction
from music21 import *

#### 1st Order Markov Chain

We first build the transition matrices for pitch and rhythm.

In [2]:
c = converter.parse(r'C:\Users\User\Documents\MuseScore4\Scores\violins.musicxml')

In [3]:
c.show('musicxml')

In [4]:
pitches, octaves, duration, offset, bars = [], [], [], [], []
data = c.recurse().notes

for tN in data:
    pitches += [[n.name for n in tN.pitches]]
    octaves += [[n.octave for n in tN.pitches]]
    duration += [tN.duration.quarterLength]
    offset += [tN.offset]
    bars += [tN.measureNumber]

# remove bars to be replicated
d = np.array([pitches, octaves, duration, offset, bars])
start = bars.index(113)
end = bars.index(137)
d = np.delete(d, [i for i in range(start, end)], axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_3608\1327628057.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  d = np.array([pitches, octaves, duration, offset, bars])


In [5]:
m = min(*octaves)[0]
M = max(*octaves)[0]
print(m, M)

3 6


In [6]:
pitches_no = np.zeros((len(pitches), 1))
dict_pitches = {'A': 0, 'A#':1, 'B-':1, 'B': 2, 'C-':2, 'B#':3, 'C': 3, 'C#':4, 'D-':4, 'D': 5, 'D#':6, 'E-':6, 'E': 7, 'E#':7, 'F':8, 'F#':9, 'G-':9, 'F##':10, 'G':10, 'G#':11, 'A-':11}

In [39]:
# build transition matrix for pitches

P = np.zeros([12*(M-m+1), 12*(M-m+1)]) # assume we stay in same range

for i in range(len(pitches)-1):
    for j in range(len(pitches[i])):
        for k in range(len(pitches[i+1])):
            ind1 = (octaves[i][j] - 3) * 12 + dict_pitches[pitches[i][j]]
            ind2 = (octaves[i+1][k] - 3) * 12 + dict_pitches[pitches[i+1][k]]
            P[ind1, ind2] += 1

P = P / np.sum(P, axis=1, keepdims=True)

C:\Users\User\AppData\Local\Temp\ipykernel_3608\1108432372.py:12: RuntimeWarning: invalid value encountered in divide
  P = P / np.sum(P, axis=1, keepdims=True)


In [40]:
P

array([[0.25      , 0.        , 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.2       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.73529412,
        0.02941176],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5       ,
        0.        ]])

In [41]:
stre = c.parts.stream()

In [42]:
# list of note and rest durations, where rests are indicated by strings

rhy = []

for n in stre.flat.notesAndRests:
    if n.measureNumber >= 113 and n.measureNumber <= 136:
        continue
    else:
        if n.isRest:
            rhy += [str(float(n.duration.quarterLength))]
        else:
            rhy += [float(n.duration.quarterLength)]

In [12]:
rhythm_set = set([float(i) for i in rhy])
mat_ind = list(rhythm_set) + [str(i) for i in rhythm_set]

In [57]:
mat_ind.index(rhy[-1])

3

In [58]:
R[14, 3]

0.043478260869565216

In [43]:
# build transition matrix for rhythm

R = np.zeros([len(mat_ind), len(mat_ind)])

for i in range(len(rhy)-1):
    ind1 = mat_ind.index(rhy[i])
    ind2 = mat_ind.index(rhy[i+1])
    R[ind1, ind2] += 1

R = R / np.sum(R, axis=1, keepdims=True)

C:\Users\User\AppData\Local\Temp\ipykernel_3608\1693021656.py:10: RuntimeWarning: invalid value encountered in divide
  R = R / np.sum(R, axis=1, keepdims=True)


In [14]:
R

array([[0.67283951, 0.32098765, 0.        , 0.        , 0.00617284,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.20833333, 0.46666667, 0.2625    , 0.00416667, 0.0125    ,
        0.        , 0.01666667, 0.00416667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.02083333,
        0.        , 0.        , 0.        , 0.00416667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.00438596, 0.29385965, 0.28508772, 0.        , 0.00438596,
        0.        , 0.37280702, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03947368, 0.        , 0.        , 0.        , 0.        ,
      

Now we run the stochastic process using the transition matrices derived.

In [15]:
def violin_note(duration, pitch = "C4"):
    # create Note object with pitch and duration
    return note.Note(pitch, quarterLength = duration*(4))

In [16]:
def create_violin_stream(time_sig = None):
    # initialize note stream for violin
    # if time signature is None, no measure splits
    if time_sig == None:
        violPart = stream.Measure()
    else:
        violPart = stream.Stream()
        violPart.timeSignature = meter.TimeSignature(time_sig)
    
    violPart.insert(0, instrument.Violin())
    return violPart

In [17]:
def append_event(duration, original_stream, rest = False, pitch = 'C4'):
    # returns a new_stream obtained by appending a rhythmical event or a rest of given duration to the original_stream
    new_stream = original_stream
    if rest:
        new_stream.append(note.Rest(quarterLength = duration*(4)))
    else:
        new_stream.append(violin_note(duration, pitch))
    return new_stream

In [18]:
def rhythm_from_sequence(durations, time_sig = None, pitch = 'C4', rhythm=None):
    # generate rhythmic stream from a list of durations. Rests are indicated by specifying a duration as a string
    if rhythm is None:
        # pass an existing stream 'rhythm' to append the durations, otherwise a new one will be created
        rhythm = create_violin_stream(time_sig = time_sig)
    for dur in durations:
        is_rest = False
        if dur != 0:
            if isinstance(dur, str):
                #if duration is given as a string, interpret and rest and turn string into a numerical value
                is_rest = True
                dur = Fraction(dur)
            
            rhythm = append_event(dur, rhythm, rest = is_rest, pitch = pitch) 
    return rhythm

In [19]:
# example
ex = rhythm_from_sequence([3/8, 1/8, '1/2', 1/4], time_sig = '4/4')
append_event(1/4, ex, pitch = 'D4')
ex.show('musicxml')

In [20]:
dict_pitches_inv = {v: k for k, v in dict_pitches.items()}

In [87]:
s = rhythm_from_sequence([], time_sig = '5/4')

bars_total = 24
beats = bars_total*5/4
new_pitch = d[0, start-1][0]
new_octave = d[1, start-1][0]
new_duration = d[2, start-1]

# generate a sequence of notes wrt the transition matrices P, R
while beats>0:
    is_rest = False
    old_pitch, old_octave, old_duration = new_pitch, new_octave, new_duration

    # pitch process
    ind1 = (old_octave - 3) * 12 + dict_pitches[old_pitch]
    ind2 = np.random.choice(np.arange(12*(M-m+1)), p=P[ind1, :])
    new_pitch = dict_pitches_inv[ind2 % 12]
    new_octave = (ind2 // 12) + 3

    # rhythm process
    ind1 = mat_ind.index(old_duration)
    ind2 = np.random.choice(len(mat_ind), p=R[ind1, :])
    new_duration = mat_ind[ind2]
    if isinstance(new_duration, str):
        is_rest = True
        new_duration = Fraction(new_duration)

    # update beats
    beats -= float(new_duration)
    if beats < 0:
        new_duration = float(new_duration) + beats
    
    if is_rest:
        new_duration = Fraction(new_duration)
    # append note to stream
    append_event(new_duration, s, rest=is_rest, pitch = new_pitch + str(new_octave))

In [86]:
s.show('musicxml')

#### 2nd Order Markov Chain

In [89]:
# 2nd order pitch transition matrix

N = 12*(M-m+1)
P = np.zeros([N**2, N])

for i in range(len(pitches)-2):
    for j in range(len(pitches[i])):
        for k in range(len(pitches[i+1])):
            for l in range(len(pitches[i+2])):
                ind1 = (octaves[i][j] - 3) * 12 + dict_pitches[pitches[i][j]]
                ind2 = (octaves[i+1][k] - 3) * 12 + dict_pitches[pitches[i+1][k]]
                ind3 = (octaves[i+2][l] - 3) * 12 + dict_pitches[pitches[i+2][l]]
                P[ind1*N + ind2, ind3] += 1

P = P / np.sum(P, axis=1, keepdims=True)

C:\Users\User\AppData\Local\Temp\ipykernel_3608\2991847980.py:15: RuntimeWarning: invalid value encountered in divide
  P = P / np.sum(P, axis=1, keepdims=True)


In [173]:
# 2nd order rhythm transition matrix

M = len(mat_ind)
R = np.zeros([M**2, M])

for i in range(len(rhy)-2):
    ind1 = mat_ind.index(rhy[i])
    ind2 = mat_ind.index(rhy[i+1])
    ind3 = mat_ind.index(rhy[i+2])
    R[ind1*M + ind2, ind3] += 1

R = R / np.sum(R, axis=1, keepdims=True)

C:\Users\User\AppData\Local\Temp\ipykernel_3608\920235247.py:12: RuntimeWarning: invalid value encountered in divide
  R = R / np.sum(R, axis=1, keepdims=True)


In [180]:
s = rhythm_from_sequence([], time_sig = '5/4')

bars_total = 24
beats = bars_total*5/4
old_pitch2, new_pitch = d[0, start-2][0], d[0, start-1][0]
old_octave2, new_octave = d[1, start-2][0], d[1, start-1][0]
old_duration2, new_duration = d[2, start-2], d[2, start-1]

# generate a sequence of notes wrt the transition matrices P, R
while beats>0:
    is_rest = False

    old_pitch1, old_pitch2 = old_pitch2, new_pitch
    old_octave1, old_octave2 = old_octave2, new_octave
    old_duration1, old_duration2 = old_duration2, new_duration

    # pitch process
    ind1 = (old_octave1 - 3) * 12 + dict_pitches[old_pitch1]
    ind2 = (old_octave2 - 3) * 12 + dict_pitches[old_pitch2]
    ind3 = np.random.choice(np.arange(N), p=P[ind1*N + ind2, :])
    new_pitch = dict_pitches_inv[ind3 % 12]
    new_octave = (ind3 // 12) + 3

    # rhythm process
    ind1 = mat_ind.index(old_duration1)
    ind2 = mat_ind.index(old_duration2)
    ind3 = np.random.choice(len(mat_ind), p=R[ind1*M + ind2, :])
    new_duration = mat_ind[ind3]
    if isinstance(new_duration, str):
        is_rest = True
        new_duration = Fraction(new_duration)

    # update beats
    beats -= float(new_duration)
    if beats < 0:
        new_duration = float(new_duration) + beats
    if is_rest:
        new_duration = Fraction(new_duration)
    # append note to stream    
    append_event(new_duration, s, rest = is_rest, pitch = new_pitch + str(new_octave))

In [181]:
s.show('musicxml')